### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 1, 22)

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-22
yesterday: 2025-01-21


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"OSD path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
OSD path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-01-21'
ORDER BY a.name



(20, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-21' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-21,2025-01-21,5.15,5.10,25049174,5.15,5.05,B,SET,-0.05,-0.97
1,AIMIRT,2025-01-13,2025-01-21,10.50,10.50,5738800,10.60,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-01-21,2025-01-21,11.90,12.10,2155940,12.20,11.90,B,SET,0.20,1.68
3,DIF,2025-01-16,2025-01-21,8.25,8.45,49659928,8.45,8.20,B,SET,0.20,2.42
4,GVREIT,2025-01-20,2025-01-21,6.65,6.55,3527934,6.65,6.45,B,SET,-0.10,-1.50
5,JMART,2025-01-21,2025-01-21,10.70,11.20,17815986,11.80,10.70,B,SET50,0.50,4.67
6,KCE,2025-01-17,2025-01-21,21.70,23.20,26237140,23.20,21.60,B,SET100,1.50,6.91
7,MCS,2025-01-21,2025-01-21,6.30,6.35,2104686,6.60,6.30,B,SET,0.05,0.79
8,NER,2025-01-17,2025-01-21,4.84,4.94,32702916,4.94,4.82,B,SET,0.10,2.07
9,SENA,2025-01-09,2025-01-21,2.20,2.18,6253440,2.24,2.16,B,SET,-0.02,-0.91


In [20]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-21,2025-01-21,5.15,5.10,"25,049,174",5.15,5.05,-0.97%,B
1,AIMIRT,2025-01-13,2025-01-21,10.50,10.50,"5,738,800",10.60,10.40,0.00%,B
2,CPNREIT,2025-01-21,2025-01-21,11.90,12.10,"2,155,940",12.20,11.90,1.68%,B
3,DIF,2025-01-16,2025-01-21,8.25,8.45,"49,659,928",8.45,8.20,2.42%,B
4,GVREIT,2025-01-20,2025-01-21,6.65,6.55,"3,527,934",6.65,6.45,-1.50%,B
5,JMART,2025-01-21,2025-01-21,10.70,11.20,"17,815,986",11.80,10.70,4.67%,B
6,KCE,2025-01-17,2025-01-21,21.70,23.20,"26,237,140",23.20,21.60,6.91%,B
7,MCS,2025-01-21,2025-01-21,6.30,6.35,"2,104,686",6.60,6.30,0.79%,B
8,NER,2025-01-17,2025-01-21,4.84,4.94,"32,702,916",4.94,4.82,2.07%,B
9,SENA,2025-01-09,2025-01-21,2.20,2.18,"6,253,440",2.24,2.16,-0.91%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file (icd_file): {icd_file}") 
print(f"god_file (god_file): {god_file}") 
print(f"osd_file (osd_file): {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file (icd_file): C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file (god_file): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file (osd_file): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(20, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-21,2025-01-21,5.15,5.10,25049174,5.15,5.05,-0.97,B,5.15,0.05,1475600,2025-01-22
1,AIMIRT,2025-01-13,2025-01-21,10.50,10.50,5738800,10.60,10.40,0.00,B,10.50,0.00,29440,2025-01-22
2,CPNREIT,2025-01-21,2025-01-21,11.90,12.10,2155940,12.20,11.90,1.68,B,12.10,0.00,1134627,2025-01-22
3,DIF,2025-01-16,2025-01-21,8.25,8.45,49659928,8.45,8.20,2.42,B,8.45,0.00,2702039,2025-01-22
4,GVREIT,2025-01-20,2025-01-21,6.65,6.55,3527934,6.65,6.45,-1.50,B,6.50,-0.05,104287,2025-01-22
5,JMART,2025-01-21,2025-01-21,10.70,11.20,17815986,11.80,10.70,4.67,B,11.10,-0.10,2047317,2025-01-22
6,KCE,2025-01-17,2025-01-21,21.70,23.20,26237140,23.20,21.60,6.91,B,22.90,-0.30,6300553,2025-01-22
7,MCS,2025-01-21,2025-01-21,6.30,6.35,2104686,6.60,6.30,0.79,B,6.25,-0.10,198500,2025-01-22
8,NER,2025-01-17,2025-01-21,4.84,4.94,32702916,4.94,4.82,2.07,B,5.00,0.06,10159575,2025-01-22
9,SENA,2025-01-09,2025-01-21,2.20,2.18,6253440,2.24,2.16,-0.91,B,2.20,0.02,539058,2025-01-22


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file (icd_file): {icd_file}") 
print(f"god_file (god_file): {god_file}") 
print(f"osd_file (osd_file): {osd_file}") 

output_file (output_file): C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file (icd_file): C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file (god_file): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file (osd_file): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [48]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [50]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-22 2025-01-22   5.10   5.15   0.98%  25,049,174   5.15   5.05 B Insert
AIMIRT   2025-01-13 2025-01-22  10.50  10.50    0.0%   5,738,800  10.60  10.40 B Update
AWC      2025-01-21 2025-01-22   3.04   3.10   1.97% 111,757,948   3.18   3.04 S Update
BCH      2025-01-21 2025-01-22  14.20  15.10   6.34%  11,559,696  15.10  14.20 S Update
CPNREIT  2025-01-21 2025-01-22  11.90  12.10   1.68%   2,155,940  12.20  11.90 B Update
DIF      2025-01-16 2025-01-22   8.25   8.45   2.42%  49,659,928   8.45   8.20 B Update
GVREIT   2025-01-20 2025-01-22   6.65   6.50  -2.26%   3,527,934   6.65   6.45 B Update
JMART    2025-01-22 2025-01-22  11.20  11.10  -0.89%  17,815,986  11.80  10.70 B Insert
JMT      2025-01-22 2025-01-22  14.90  14.80  -0.67%  72,617,738  15.70  14.30 S Insert
KCE      2025-01-22 2025-01-22  

In [52]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [54]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-22' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [56]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [58]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-01-22,5.10,5.15,"25,049,174",5.15,5.05,B,SET,0.05,0.98%
1,AIMIRT,2025-01-13,2025-01-22,10.50,10.50,"11,477,600",10.60,10.40,B,SET,0.00,0.00%
2,CPNREIT,2025-01-21,2025-01-22,11.90,12.10,"4,311,880",12.20,11.90,B,SET,0.20,1.68%
3,DIF,2025-01-16,2025-01-22,8.25,8.45,"99,319,856",8.45,8.20,B,SET,0.20,2.42%
4,GVREIT,2025-01-20,2025-01-22,6.65,6.50,"7,055,868",6.65,6.45,B,SET,-0.15,-2.26%
5,JMART,2025-01-22,2025-01-22,11.20,11.10,"17,815,986",11.80,10.70,B,SET50,-0.10,-0.89%
6,KCE,2025-01-22,2025-01-22,23.20,22.90,"26,237,140",23.20,21.60,B,SET100,-0.30,-1.29%
7,MCS,2025-01-22,2025-01-22,6.35,6.25,"2,104,686",6.60,6.25,B,SET,-0.10,-1.57%
8,NER,2025-01-17,2025-01-22,4.84,5.00,"65,405,832",5.00,4.82,B,SET,0.16,3.31%
9,SENA,2025-01-22,2025-01-22,2.18,2.20,"6,253,440",2.24,2.16,B,SET,0.02,0.92%


In [60]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [62]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
18,TVO,2025-01-02,2025-01-22,22.90,21.00,"31,217,464",23.20,21.00,S,SET,-1.90,-8.30%


In [64]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
13,BCH,2025-01-21,2025-01-22,14.20,15.10,"23,119,392",15.10,14.20,S,SET100,0.90,6.34%


In [66]:
conlite.close()